# ETL Pipeline

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# loading messages dataset
df_messages = pd.read_csv('../data/01_raw/messages.csv')
df_messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# loading categories dataset
df_categories = pd.read_csv('../data/01_raw/categories.csv')
df_categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [4]:
print(df_categories.iloc[200]['categories'])

related-1;request-1;offer-0;aid_related-1;medical_help-1;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-1;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-1;other_aid-1;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-1


In [5]:
# merge datasets
df = df_messages.merge(df_categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [6]:
# extract a list of new column names for categories from the first row.
categories_cols = [col.split('-')[0] for col in df['categories'].iloc[0].split(';')]
# create a dataframe of the 36 individual category columns
df_expanded_cetegories = df['categories'].str.split(';',expand=True)
# rename the columns of `categories`
df_expanded_cetegories.columns = categories_cols
# set each value to be the last character of the string and convert column from string to numeric
for col in categories_cols:
    df_expanded_cetegories[col] = df_expanded_cetegories[col].apply(lambda s: int(s.split('-')[-1]))

df_expanded_cetegories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# concatenate the extracted and cleaned categories with the former dataset
df = pd.concat([df, df_expanded_cetegories], axis=1)
# drop categories columns (since it`s content is alredy on the concatenated columns)
df.drop('categories', axis=1, inplace=True)

In [8]:
# checking duplicates
print(f'There are {sum(df.duplicated())} duplicates on {df.shape[0]} rows')
# drop duplicates
print('Drop duplicates')
df.drop_duplicates(inplace=True)
print(f'Now, there are {sum(df.duplicated())} duplicates on {df.shape[0]} rows')

There are 170 duplicates on 26386 rows
Drop duplicates
Now, there are 0 duplicates on 26216 rows


In [9]:
# create SQLite DB
conn = sqlite3.connect('../data/02_trusted/worldbank.db')

In [10]:
# export clean dataset to SQLite
df.to_sql('messages_dataset',con=conn,if_exists='replace')